In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
%matplotlib qt

!mkdir RRT_Path
!mkdir
class nonhn_RRT:
    def __init__(self,start_config,goal,v_init,w_init):
        (x0,y0,theta0) = start_config
        self.xg = goal[0]
        self.yg = goal[1]
        self.x = []
        self.y = []
        self.theta = []
        self.x.append(x0)
        self.y.append(y0)
        self.theta.append(theta0)
        self.vc = [] #platform centre v list
        self.wc = [] #platform centre w list
        self.vc.append(v_init)
        self.wc.append(w_init)
        
        #declared variables
        self.node_distances=[]
        self.paths=[[]]
        self.tree=[]
        self.tree.append([x0,y0,theta0,-1])
        self.step_size=5
        self.goal_radius=2 # goal marker size - cart radius
        self.previous_slope=theta0
        
        #rrt non-holonomic variables
        self.vc_min=10
        self.vc_max= 20
        self.wc_min=-2
        self.wc_max= 2
        self.random_vc_wc_size=3
        self.path_tree = []
        self.cart_base = 3

        
    # implement non-holonomic RRT to find platform centre velocities
    def on_goal(self,point_xy):
        goal_center = [self.xg,self.yg]
        if(self.inside_circle(goal_center,self.goal_radius,point_xy)):
            return True
        else:
            return False
        
    def goal_reached_check(self,point_xy):
        if(self.on_goal(point_xy)):
            return True
        else:
            return False
    
    def compute_distances(self,random_point_xy, tree):
        self.node_distances.clear()
        for node_xytheta in tree:
            l2_norm = np.linalg.norm(node_xytheta[0:2] - random_point_xy)
            self.node_distances.append(l2_norm)
            
       
    def slope(self,node_xy, point_xy):
        num = (point_xy[1]-node_xy[1])
        den = (point_xy[0]-node_xy[0])
        #print("numerator:",num)
        #print("denominator:",den)
        #print("slope of propagation:",num/den)
        if((den== 0) or (num==0 and den==0)):
            if((point_xy[0]==0) or (point_xy[0]==0 and point_xy[1]==0)):
                return self.previous_slope
            else:
                self.previous_slope=point_xy[1]/point_xy[0]
                return point_xy[1]/point_xy[0]
        else:
            self.previous_slope=point_xy[1]/point_xy[0]
            return num/den
        
        
    def compute_node_xy(self,nearest_node, random_point_xy, tree):
        prop_angle = np.arctan(self.slope(tree[nearest_node], random_point_xy))
        #print("propagation angle:", prop_angle)
        prop_node_xy = []
        prop_x=int(tree[nearest_node][0])+int(self.step_size*np.cos(prop_angle))
        #print("propagation x:", prop_x)
        prop_y=int(tree[nearest_node][1])+int(self.step_size*np.sin(prop_angle))
        #print("propagation y:", prop_y)
        prop_node_xy.append(prop_x)
        prop_node_xy.append(prop_y)
        #print("propagated node:", prop_node_xy)
        return prop_node_xy
    
    def inside_circle(self,center_xy,radius, point_xy):
        if(((point_xy[0]-center_xy[0])**2)+((point_xy[1]-center_xy[1])**2) <= radius**2):
            return True
        else:
            return False
    
    def on_obstacle(self, point_xy):
        is_on_obstacle = False
        b = self.cart_base
        center_xy=[[20,20],[90,70],[77,82]]
        radius=[7,2,3]
        #checking rectangular obstacles
        if ((40-b<=point_xy[0]<=60+b) and (62-b<=point_xy[1]<=100+b)):
            is_on_obstacle = True
            return is_on_obstacle
        #checking the first rectangle
        if ((70-b<=point_xy[0]<=80+b) and (40-b<=point_xy[1]<=60+b)):
            is_on_obstacle = True
            return is_on_obstacle
        #checking the second rectangle
        if ((70-b<=point_xy[0]<=80+b) and (40-b<=point_xy[1]<=60+b)):
            is_on_obstacle = True
            return is_on_obstacle
        #checking the third rectangle
        if ((40-b<=point_xy[0]<=60+b) and (0<=point_xy[1]<=38+b)):
            is_on_obstacle = True
            return is_on_obstacle
        if (self.inside_circle(center_xy[0],radius[0]+b,point_xy)):
            is_on_obstacle = True
            return is_on_obstacle
        if (self.inside_circle(center_xy[1],radius[1]+b,point_xy)):
            is_on_obstacle = True
            return is_on_obstacle
        if (self.inside_circle(center_xy[2],radius[2]+b,point_xy)):
            is_on_obstacle = True
            return is_on_obstacle
        """if (self.inside_circle(center_xy[3],radius[3]+b,point_xy)):
            is_on_obstacle = True
            return is_on_obstacle"""
        if((point_xy[0]<=0 or point_xy[0]>=100) or (point_xy[1]<=0 or point_xy[1]>=100)):
            is_on_obstacle = True
            return is_on_obstacle
        else:
            return False

    
    def check_line_obstacle_free(self,tree,nearest_node, random_point_xy):
        #find minimum number of integer points on the line using the smallest 
        #abs diff coordinate between nearest node and random point
        #abs_diff = [abs(tree[nearest_node][0]-random_point_xy[0]),abs(tree[nearest_node][1]-random_point_xy[1])]
        min_div = 50
        l = math.sqrt((tree[nearest_node][1]-random_point_xy[1])**2 + (tree[nearest_node][0]-random_point_xy[0])**2)
        r=l/min_div
        line_xy=[]
        point_xy_on_line=[]
        
        m=self.slope(tree[nearest_node][0:2],random_point_xy)
        theta = np.arctan(m)
        
        for i in range(min_div):
            point_xy_on_line=[tree[nearest_node][0]+r*i*np.cos(theta),
                           tree[nearest_node][1]+r*i*np.sin(theta)]
            line_xy.append(point_xy_on_line)
            
        #debug output & plot
        #print(line_xy)
        lin_x = [xy[0] for xy in line_xy]
        lin_y = [xy[1] for xy in line_xy]
        #plt.gcf().gca().plot(lin_x,lin_y,linewidth=0.1)
        #plt.gcf().gca().plot(random_point_xy[0],random_point_xy[1], 'y*',markersize=1)
        #plt.show()
        
        
        for point_xy in line_xy:
            #if the point satisfies the condition of the obstacle return false
            if(self.on_obstacle(point_xy)):
                return False
            #else return true
            else:
                return True
            
    def compute_final_paths(self, tree, nearest_node, random_vc_wc):
        time_step = 0.1
        integration_time = 1
        x_list=[]
        y_list=[]
        theta_list=[]
        xytheta_list=[]
        final_paths=[]
        for i in range(len(random_vc_wc)):
            x_list.clear()
            y_list.clear()
            theta_list.clear()
            xytheta_list.clear()
            x=tree[nearest_node][0]
            #x_list.append(x)
            y=tree[nearest_node][1]
            #y_list.append(y)
            theta=tree[nearest_node][2]
            #theta_list.append(theta)
            #xytheta_list.append([x,y,theta])
            #print("parent_node x,y,theta:",x,y,theta)
            for j in range(int(integration_time/time_step)):
                if(self.on_obstacle([x+random_vc_wc[i][0]*np.cos(theta+random_vc_wc[i][1]*time_step)*time_step,
                                     y+random_vc_wc[i][0]*np.sin(theta+random_vc_wc[i][1]*time_step)*time_step])): #break out of that path
                    break
                else: #compute that path
                    theta+=random_vc_wc[i][1]*time_step
                    theta_list.append(theta)
                    x+=random_vc_wc[i][0]*np.cos(theta)*time_step
                    x_list.append(x)
                    y+=random_vc_wc[i][0]*np.sin(theta)*time_step
                    y_list.append(y)
                    xytheta_list.append([x,y,theta]) #for final paths list to be returned
                    #print('x,y,theta,vc,wc :',x,y,theta,random_vc_wc[i][0],random_vc_wc[i][1])
            final_paths.append(xytheta_list)
            plt.gcf().gca().plot(x_list,y_list,linewidth=0.2)
            #plt.show()
        return final_paths
    
    def compute_distances_final_paths(self, final_paths, random_point_xy):
        final_path_point_distances=[]
        l2_norm = 0
        for path in final_paths:
            for point_xy in path:
                if self.on_obstacle(point_xy):
                    l2_norm = float('inf')
                else :
                    l2_norm = 0
                    
            if(l2_norm == 0):
                if(len(path)>0):
                    final_path_point = path[-1]
                    l2_norm = np.linalg.norm(final_path_point[0:2]-random_point_xy)
                else:
                    l2_norm=0
                      
            
            final_path_point_distances.append(l2_norm)
            
        return final_path_point_distances
    
    
    def check_path_line_obstacle_free(self, final_paths, nearest_final_path_point, random_point_xy):
        #the required point is final_paths[nearest_final_path_point][-1]
        min_div = 50
        l = math.sqrt((final_paths[nearest_final_path_point][-1][1]-random_point_xy[1])**2 + (final_paths[nearest_final_path_point][-1][0]-random_point_xy[0])**2)
        r=l/min_div
        line_xy=[]
        point_xy_on_line=[]
        #constructing the line
        m=self.slope(final_paths[nearest_final_path_point][-1][0:2],random_point_xy)
        theta = np.arctan(m)
        
        for i in range(1,min_div):
            point_xy_on_line=[final_paths[nearest_final_path_point][-1][0]+r*i*np.cos(theta),
                           final_paths[nearest_final_path_point][-1][1]+r*i*np.sin(theta)]
            line_xy.append(point_xy_on_line)
            
        #debug output & plot
        #print(line_xy)
        lin_x = [xy[0] for xy in line_xy]
        lin_y = [xy[1] for xy in line_xy]
        #plt.gcf().gca().plot(lin_x,lin_y,linewidth=0.1)
        #plt.gcf().gca().plot(random_point_xy[0],random_point_xy[1], 'r*',markersize=1)
        #plt.show()
        
        #checking if the points on the line are on an obstacle
        for point_xy in line_xy:
            #if the point satisfies the condition of the obstacle return false
            if(self.on_obstacle(point_xy)):
                return False
            #else return true
            else:
                return True
            
    def check_path_curve_obstacle_free(self,final_paths,nearest_final_path_point):
        for point_xy in final_paths[nearest_final_path_point][0:2]:
            #if the point satisfies the condition of the obstacle return false
            if(self.on_obstacle(point_xy)):
                return False
            #else return true
            else:
                return True
            
        
                 
    def propagate_tree(self,tree, node_distances, random_point_xy):
        #pick the nearest node in the tree
        nearest_node = node_distances.index(min(node_distances))
        final_paths = []
        #print("nearest_node(number):",nearest_node)
        #compute a new kinematically compliant node biased in the direction of propagation via non-holonomic constraints
        #if the line joining the nearest node and the random point is not crossing an object
        if(self.check_line_obstacle_free(tree, nearest_node, random_point_xy)):
            #randomly sample v,w vector of size random_vc_wc 
            random_vc_wc = np.random.uniform(low = [self.vc_min,self.wc_min],high=[self.vc_max,self.wc_max],size=[self.random_vc_wc_size,2])
            #compute final paths for those vc and wc
            final_paths = self.compute_final_paths(tree, nearest_node, random_vc_wc)
            
            #compute distances to the random point from the final path point of each final path
            final_path_point_distances = self.compute_distances_final_paths(final_paths, random_point_xy)
            #pick the path with the nearest final path point
            nearest_final_path_point = final_path_point_distances.index(min(final_path_point_distances))
            #if the line joining the nearest finalpath point and the random point is not crossing an object
            if(self.check_path_curve_obstacle_free(final_paths,nearest_final_path_point)):
                if(self.check_path_line_obstacle_free(final_paths, nearest_final_path_point, random_point_xy)):
                    node_xytheta = [final_paths[nearest_final_path_point][-1][0],
                                    final_paths[nearest_final_path_point][-1][1],
                                    final_paths[nearest_final_path_point][-1][2]]
                    final_sub_path_x_list= [final_paths[nearest_final_path_point][i][0] for i in range(len(final_paths[nearest_final_path_point]))] # extracting all x coordinates of the path with i
                    final_sub_path_y_list= [final_paths[nearest_final_path_point][i][1] for i in range(len(final_paths[nearest_final_path_point]))] # extracting all y coordinates of the path with i
                    final_sub_path_theta_list= [final_paths[nearest_final_path_point][i][2] for i in range(len(final_paths[nearest_final_path_point]))]
                    plt.gcf().gca().plot(final_sub_path_x_list,final_sub_path_y_list, color='orange',linewidth=1)
                    #write code to add to robot stage path variables self.x and self.y
                    self.path_tree.append([final_sub_path_x_list,final_sub_path_y_list,final_sub_path_theta_list,random_vc_wc[nearest_final_path_point-1],nearest_node])
                    self.tree.append([node_xytheta[0],node_xytheta[1],node_xytheta[2],nearest_node])
                    if(len(self.tree)%10 == 0):
                        plt.savefig('RRT_Tree/snap%d.png'%int(len(self.tree)/10)) #save only if the tree branches out once every 10 nodes
                        pass
                else:
                    pass
            else:
                pass
            
            #functions to define
            """
            1.compute_final_paths - returns final paths - completed
            2.compute_distances_final_paths - returns final path point distances - completed
            3.check_path_line_obstacle_free - checks if the line connecting the final path point of the nearest path is obstacle free - completed
            3.check_path_curve_obstacle_free - checks if the path points of the nearest final path curve are obstacle free - completed
            """    
        else:
            pass
        #print("propagated node_xy:", node_xy)
        #append the tree data structure with the latest node coordinates and its parent node
        #print("current_node_number: ", len(tree)-1)
        #print("TREE:",self.tree)
        #print(" ")
        #print(" ")
        self.compute_path(tree)
        
    def propagate_tree_init(self, tree, vc_wc_init):
            time_step = 0.1
            integration_time = 1
            x_list=[]
            y_list=[]
            theta_list=[]
            xytheta_list=[]
            x=tree[0][0]
            #x_list.append(x)
            y=tree[0][1]
            #y_list.append(y)
            theta=tree[0][2]
            #theta_list.append(theta)
            xytheta_list.append([x,y,theta])
            for j in range(int(integration_time/time_step)):
                theta+=vc_wc_init[1]*time_step
                theta_list.append(theta)
                x+=vc_wc_init[0]*np.cos(theta)*time_step
                x_list.append(x)
                y+=vc_wc_init[0]*np.sin(theta)*time_step
                y_list.append(y)
                xytheta_list.append([x,y,theta])
                #print('x,y,theta,vc,wc :',x,y,theta,vc_wc_init[0],vc_wc_init[1])
            plt.gcf().gca().plot(x_list,y_list, color='orange',linewidth=1)
            self.path_tree.append([x_list,y_list,theta_list,vc_wc_init,0])
            self.tree.append([x,y,theta,0])
        

        
    def compute_path(self,tree):
        
        i=len(tree)-1
        path=[]

        while i>0 :
            path.append(i)
            i=tree[i][3]
        
        self.paths.append(path)
        
        

    
    def showpath(self):
        """
        Create a new directory (say nonholonomic_path) and save snapshots of the robot moving along the
        trajectory found by RRT at every step with the step index.
        e.g: snap1.png, snap2.png ...
        """
        #extract goal path
        goal_path=self.paths[-1]
        
        #print("DISTANCES",self.node_distances)

        print("GOAL PATH:",goal_path)
        #print("PATHS:",self.paths)
        print("")
        x_goal=[]
        y_goal=[]
        x_goal.append(self.xg)
        y_goal.append(self.yg)
        #print(self.path_tree[len(self.path_tree)-1])
        #print(len(self.path_tree))
        
        for node_number in goal_path:
            x=self.tree[node_number][0]
            y=self.tree[node_number][1]
            #print("[x,y]: ", x,y)
            plt.gcf().gca().plot(x,y,'co',markersize=5)
            x_goal.append(x)
            y_goal.append(y)
        x_goal.append(self.x[0])
        y_goal.append(self.y[0])
        
        
        k=0
        path_node_number=len(self.path_tree)-1
        consolidated_path_xc = []
        consolidated_path_yc = []
        consolidated_path_thetac=[]
        consolidated_path_vc_wc=[]
        while(path_node_number >=0):
            x_list=self.path_tree[path_node_number][0]
            y_list=self.path_tree[path_node_number][1]
            theta_list=self.path_tree[path_node_number][2]
            vc_wc=self.path_tree[path_node_number][3]
            x_list.reverse()
            y_list.reverse()
            theta_list.reverse()
            #no need to reverse vc_wc as it is constant for a path length of self path tree
            for xc in x_list:
                consolidated_path_xc.append(xc)
            for yc in y_list:
                consolidated_path_yc.append(yc)
            for thetac in theta_list:
                consolidated_path_thetac.append(thetac)
                consolidated_path_vc_wc.append(vc_wc)

            
            path_node_number=self.path_tree[path_node_number][4]-1
        
        #print(consolidated_path_xc)
        #print("VC_WC: ",consolidated_path_vc_wc)
        consolidated_path_xc.reverse()
        consolidated_path_yc.reverse()
        consolidated_path_thetac.reverse()
        consolidated_path_vc_wc.reverse()
        #print("[x,y,VC_WC]: ",consolidatedconsolidated_path_vc_wc)
        #for xc_i in range(len(consolidated_path_xc)):
            #print("[x,y,theta,vc,wc]: ",consolidated_path_xc[xc_i],consolidated_path_yc[xc_i],consolidated_path_thetac[xc_i],
            #      consolidated_path_vc_wc[xc_i][0],consolidated_path_vc_wc[xc_i][1])
            #plt.gcf().gca().plot(consolidated_path_xc[:xc_i],consolidated_path_yc[:xc_i],color='brown',linewidth=1)
            #plt.savefig('RRT_Path/snap%d.png'%xc_i)


            
        plt.xlim(-10,110)
        plt.ylim(-10,110)
        plt.show()
        
        
        return [consolidated_path_xc,consolidated_path_yc,consolidated_path_thetac,consolidated_path_vc_wc]
    
    
        
        
        
    def showtree(self):
        """
        Create a new directory (say nonholonomic_tree) and save snapshots of the evolution of the RRT tree
        at every step with the step index.
        e.g: snap1.png, snap2.png ...
        """
        
        i=0
        k=0
        last_node = 0
        current_branch_point = [self.tree[-1][0],self.tree[-1][1]] # used to check goal reached condition
        #print(self.goal_reached_check(current_branch_point))
        self.propagate_tree_init(self.tree,[self.vc[0],self.wc[0]])
        plt.xlim(-10,110)
        plt.ylim(-10,110)
        while(self.goal_reached_check(current_branch_point) is False):
        #while(i<100):
            #generate 5 random linear and angular velocities based on minimum and maximum platform center velocities
            """note that the minimum and maximum platform center velocities can be calculated using minimum and maximum
            left and right wheel velocities"""
            #pick a random point from the robot's obstacle free configuration space
            random_point_xy = [101,101] #initializing the random point on obstacle to make sure the loop works to pick a non-obstacle random point
            while(self.on_obstacle(random_point_xy)):
                #random_point_xy = np.random.randint(low = 0, high = 100, size=2)
                distance_of_last_node = np.linalg.norm(np.array(self.tree[-1][0:2])-np.array([self.xg,self.yg]))
                random_point_xy = np.random.normal(loc=100,scale=distance_of_last_node,size=2)
            #pick any random point in the robot's configuration space
            #random_point_xy = np.random.randint(low = 0, high = 100, size=2)
            #compute distances of all the nodes of the tree to the random point
            self.compute_distances(random_point_xy, self.tree)
            
            #propogate the tree in the direction of nearest non-holonomic constraint satisfying randomly sample control
            
            self.propagate_tree(self.tree,self.node_distances,random_point_xy)
            i+=1
            current_branch_point = [self.tree[-1][0],self.tree[-1][1]]

                

            
        plt.savefig('RRT_Tree/snap%d.png'%int(len(self.tree)/10)) # save the last goal reached frame    
        plt.show()
                
        
    def test_env(self):
        """
        Function to generate test environment
        """
        # the vertices for each obstacle are given in the clockwise order starting from lower left
        obst_vertex_x = [40,40,60,60,70,70,80,80,40,40,60,60] 
        obst_vertex_y = [62,100,100,62,40,60,60,40, 0,38,38, 0]
        plt.plot(self.xg,self.yg,'g*',markersize=20)
        plt.plot(self.x[0],self.y[0],'b*',markersize=20)
        for i in range(1,4): 
            x = obst_vertex_x[4*(i-1)]
            y = obst_vertex_y[4*(i-1)]
            width = obst_vertex_x[4*(i-1)+2] - obst_vertex_x[4*(i-1)]
            height = obst_vertex_y[4*(i-1)+2] - obst_vertex_y[4*(i-1)]
            rect = plt.Rectangle((x,y),width,height,linewidth=1,color='r')
            plt.gca().add_patch(rect)
           
        #centre positions of circular obstacles
        circle_x = [20,90,77]
        circle_y = [20,70,82]
        radii = [7,2,3]

        for i in range(3): #plotting  circles
            circle = plt.Circle((circle_x[i],circle_y[i]),radii[i], color='r')
            plt.gca().add_patch(circle)

        plt.gca().set_aspect('equal', adjustable='box')
        plt.show()
        
        
    def visualize_wheels(self):
        self.showtree()
        consolidated_paths = self.showpath()
        xc=self.x[0]
        xc_list=[]
        yc=self.y[0]
        yc_list=[]
        time_step=0.1
        
        xl=self.x[0]
        xl_list=[]
        yl=self.y[0]+self.cart_base
        yl_list=[]

        
        xr=self.x[0]
        xr_list=[]
        yr=self.y[0]-self.cart_base
        yr_list=[]
        
        vl=0
        vl_list=[]
        vr=0
        vr_list=[]

        
        
        for i in range(len(consolidated_paths[2])):
            vc=consolidated_paths[-1][i][0]
            xc+=vc*np.cos(consolidated_paths[2][i])*time_step
            yc+=vc*np.sin(consolidated_paths[2][i])*time_step
            xc_list.append(xc)
            yc_list.append(yc)
            plt.gcf().gca().plot(xc_list,yc_list,color='brown',linewidth=1)
            
            wc= consolidated_paths[-1][i][1]
            rc= vc/wc
            
            vl= (rc-self.cart_base)*wc
            vr= (rc+self.cart_base)*wc
            
            xl+=vl*np.cos(consolidated_paths[2][i])*time_step
            xl_list.append(xl)
            xr+=vr*np.cos(consolidated_paths[2][i])*time_step
            xr_list.append(xr)
            
            
            
            yl+=vl*np.sin(consolidated_paths[2][i])*time_step
            yl_list.append(yl)
            yr+=vr*np.sin(consolidated_paths[2][i])*time_step
            yr_list.append(yr)
            
            plt.gcf().gca().plot(xl_list,yl_list,color='brown',linewidth=1, linestyle='dashed')
            plt.gcf().gca().plot(xr_list,yr_list,color='brown',linewidth=1, linestyle='dashed')
            
            plt.savefig('RRT_Path/snap%d.png'%i)
    

A subdirectory or file RRT_Path already exists.
The syntax of the command is incorrect.


In [2]:
goal = (100,100)
start_config =(0,1,0)
vinit = 5
winit = 0.1
nn_RRT = nonhn_RRT(start_config,goal,vinit,winit)
nn_RRT.test_env()

In [ ]:
nn_RRT.visualize_wheels()

GOAL PATH: [584, 574, 568, 530, 504, 494, 485, 484, 443, 394, 378, 267, 258, 250, 161, 90, 21, 20, 5, 4, 2, 1]



In [4]:
#nn_RRT.showtree()

In [5]:
0.10000000000000003+0.1*-1.1488942416814343

-0.0148894241681434

In [6]:
4.990380276334649+17.709227283043163*math.cos(-0.0148894241681434)*0.1

6.761106705950866

In [7]:
-0.0148894241681434

-0.0148894241681434

In [8]:
#nn_RRT.showpath()

In [9]:
0.10000000000000003+-0.23227248054958727*0.1 #new theta comes from old theta

0.07677275194504131

In [10]:
4.990380276334649+17.177278957710126*math.cos(0.07677275194504131)*0.1

6.703048466295139